In [27]:
import os
import sys
import pickle
import numpy as np
import tensorflow as tf

from sklearn import svm
from sklearn import tree
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier, BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from sklearn.decomposition import PCA 
from sklearn.utils import shuffle
from sklearn.externals import joblib

sys.path.insert(0, "/home/anonymous/Desktop/eeg_sampling/modeling")
from dataio import readdata, readlabels, writedata

In [28]:
def weight_avg(a, b, c):
    d = []
    for i in range(0,len(a)):
        bins = [0, 0, 0]
        bins[a[i]] += 1
        bins[b[i]] += 1
        bins[c[i]] += 1
        d.append(bins.index(max(bins)))
    return(d)

In [29]:
#Read training
train_dataset_right = readdata("./../curated/train_dataset_right")
train_dataset_left = readdata("./../curated/train_dataset_left")
train_dataset_avg = readdata("./../curated/train_dataset_avg")
train_labels = readlabels("./../curated/train_labels")

#Read test
test_dataset_right = readdata("./../curated/test_dataset_right")
test_dataset_left = readdata("./../curated/test_dataset_left")
test_dataset_avg = readdata("./../curated/test_dataset_avg")
test_labels = readlabels("./../curated/test_labels")

print('Training:', train_dataset_right.shape, train_dataset_left.shape, train_dataset_avg.shape, train_labels.shape)
print('Testing:', test_dataset_right.shape, test_dataset_left.shape, test_dataset_avg.shape, test_labels.shape)

Training: (451, 241, 1) (451, 241, 1) (451, 241, 1) (451,)
Testing: (115, 241, 1) (115, 241, 1) (115, 241, 1) (115,)


In [30]:
n_input = 241*1 # EEG data input (16 Hz range * 480 sample points)
n_classes = 3 # EEG total classes ("nothing", "up", "down", "left", "right")

In [31]:
train_dataset_right.shape = (train_dataset_right.shape[0], n_input)
train_dataset_left.shape = (train_dataset_left.shape[0], n_input)
train_dataset_avg.shape = (train_dataset_avg.shape[0], n_input)

test_dataset_right.shape = (test_dataset_right.shape[0], n_input)
test_dataset_left.shape = (test_dataset_left.shape[0], n_input)
test_dataset_avg.shape = (test_dataset_avg.shape[0], n_input)

print(train_dataset_right.shape)
print(train_dataset_left.shape)
print(train_dataset_avg.shape)

print(test_dataset_right.shape)
print(test_dataset_left.shape)
print(test_dataset_avg.shape)

(451, 241)
(451, 241)
(451, 241)
(115, 241)
(115, 241)
(115, 241)


In [39]:
#KNN
knn = KNeighborsClassifier(
    algorithm="auto", 
    weights="uniform", 
    n_neighbors=30)

knn.fit(train_dataset_right, train_labels)
knn_pred = knn.predict(test_dataset_right)
knn_acc = accuracy_score(test_labels, knn_pred)
print("Knn Acc Right: ", knn_acc)

knn.fit(train_dataset_left, train_labels)
knn_pred = knn.predict(test_dataset_left)
knn_acc = accuracy_score(test_labels, knn_pred)
print("Knn Acc Left: ", knn_acc)

knn.fit(train_dataset_avg, train_labels)
knn_pred = knn.predict(test_dataset_avg)
knn_acc = accuracy_score(test_labels, knn_pred)
print("Knn Acc Avg: ", knn_acc)

Knn Acc Right:  0.426086956522
Knn Acc Left:  0.391304347826
Knn Acc Avg:  0.321739130435


In [40]:
#LDA
lda = LinearDiscriminantAnalysis(
    solver="svd",
    store_covariance=False,
    tol = 0.0001)

lda.fit(train_dataset_right, train_labels)
lda_pred = lda.predict(test_dataset_right)
lda_acc = accuracy_score(test_labels, lda_pred)
print("Lda Acc Right: ", lda_acc)

lda.fit(train_dataset_left, train_labels)
lda_pred = lda.predict(test_dataset_left)
lda_acc = accuracy_score(test_labels, lda_pred)
print("Lda Acc Left: ", lda_acc)

lda.fit(train_dataset_avg, train_labels)
lda_pred = lda.predict(test_dataset_avg)
lda_acc = accuracy_score(test_labels, lda_pred)
print("Lda Acc Avg: ", lda_acc)

Lda Acc Right:  0.339130434783
Lda Acc Left:  0.417391304348
Lda Acc Avg:  0.339130434783


In [41]:
#SVM
svm_mod = svm.LinearSVC(
    C=1.0,
    penalty="l2",
    loss="squared_hinge",
    tol=0.0001)

svm_mod.fit(train_dataset_right, train_labels)
svm_pred = svm_mod.predict(test_dataset_right)
svm_acc = accuracy_score(test_labels, svm_pred)
print("Svm Acc Right: ", svm_acc)

svm_mod.fit(train_dataset_left, train_labels)
svm_pred = svm_mod.predict(test_dataset_left)
svm_acc = accuracy_score(test_labels, svm_pred)
print("Svm Acc Left: ", svm_acc)

svm_mod.fit(train_dataset_avg, train_labels)
svm_pred = svm_mod.predict(test_dataset_avg)
svm_acc = accuracy_score(test_labels, svm_pred)
print("Svm Acc Avg: ", svm_acc)

Svm Acc Right:  0.321739130435
Svm Acc Left:  0.391304347826
Svm Acc Avg:  0.373913043478


In [42]:
#Random Forests
forest = ExtraTreesClassifier(n_estimators = 1000)

forest.fit(train_dataset_right, train_labels)
forest_pred = forest.predict(test_dataset_right)
forest_acc = accuracy_score(test_labels, forest_pred)
print("For Acc Right: ", forest_acc)

forest.fit(train_dataset_left, train_labels)
forest_pred = forest.predict(test_dataset_left)
forest_acc = accuracy_score(test_labels, forest_pred)
print("For Acc Left: ", forest_acc)

forest.fit(train_dataset_avg, train_labels)
forest_pred = forest.predict(test_dataset_avg)
forest_acc = accuracy_score(test_labels, forest_pred)
print("For Acc Avg: ", forest_acc)

For Acc Right:  0.330434782609
For Acc Left:  0.4
For Acc Avg:  0.373913043478


In [43]:
#Ada
ada = AdaBoostClassifier(n_estimators = 50)

ada.fit(train_dataset_right, train_labels)
ada_pred = ada.predict(test_dataset_right)
ada_acc = accuracy_score(test_labels, ada_pred)
print("Ada Acc Right: ", ada_acc)

ada.fit(train_dataset_left, train_labels)
ada_pred = ada.predict(test_dataset_left)
ada_acc = accuracy_score(test_labels, ada_pred)
print("Ada Acc Left: ", ada_acc)

ada.fit(train_dataset_avg, train_labels)
ada_pred = ada.predict(test_dataset_avg)
ada_acc = accuracy_score(test_labels, ada_pred)
print("Ada Acc Avg: ", ada_acc)

Ada Acc Right:  0.391304347826
Ada Acc Left:  0.391304347826
Ada Acc Avg:  0.347826086957


In [44]:
#Logistic
lgr = linear_model.LogisticRegression()

lgr.fit(train_dataset_right, train_labels)
lgr_pred_right = lgr.predict(test_dataset_right)
lgr_acc = accuracy_score(test_labels, lgr_pred_right)
print("Lgr Acc Right: ", lgr_acc)

lgr.fit(train_dataset_left, train_labels)
lgr_pred_left = lgr.predict(test_dataset_left)
lgr_acc = accuracy_score(test_labels, lgr_pred_left)
print("Lgr Acc Left: ", lgr_acc)

lgr.fit(train_dataset_avg, train_labels)
lgr_pred_avg = lgr.predict(test_dataset_avg)
lgr_acc = accuracy_score(test_labels, lgr_pred_avg)
print("Lgr Acc Avg: ", lgr_acc)

lgr_weighted_avg = weight_avg(lgr_pred_right, lgr_pred_left, lgr_pred_avg)
lgr_acc = accuracy_score(test_labels, lgr_weighted_avg)
print("Lgr Acc WAvg:", lgr_acc)

Lgr Acc Right:  0.339130434783
Lgr Acc Left:  0.365217391304
Lgr Acc Avg:  0.313043478261
Lgr Acc WAvg: 0.313043478261


In [38]:
#PCA 
pca = PCA(n_components=25)

train_dataset_right = pca.fit_transform(train_dataset_right)
test_dataset_right = pca.fit_transform(test_dataset_right)
print(train_dataset_right.shape)
print(test_dataset_right.shape)

train_dataset_left = pca.fit_transform(train_dataset_left)
test_dataset_left = pca.fit_transform(test_dataset_left)
print(train_dataset_left.shape)
print(test_dataset_left.shape)

train_dataset_avg = pca.fit_transform(train_dataset_avg)
test_dataset_avg = pca.fit_transform(test_dataset_avg)
print(train_dataset_avg.shape)
print(test_dataset_avg.shape)

(451, 25)
(115, 25)
(451, 25)
(115, 25)
(451, 25)
(115, 25)


In [ ]:
print(train_dataset_right.shape)
print(test_dataset_right.shape)